In [1]:
from glob import glob
import tensorflow as tf
from tensorflow.train import Example, Features, Feature, BytesList, Int64List, FloatList
from tensorflow.io import TFRecordWriter
import re
import sys
sys.path.append("../../")

import tfutils
import pandas as pd

from scipy.stats import boxcox
import numpy as np
from cityhash import CityHash32

In [2]:
cat_feature = [
    'svr_region',
    'trigger_type',
    'language'
]

common_feature_boolean = [
    "role_create_os_android",
    "role_create_os_ios",
    "if_accounts_first_role",
    "if_pruchase_active_in_1day",
    "if_pruchase_active_in_30day",
    "if_pruchase_active_in_3day",
    "if_pruchase_active_in_7day",
    "is_acc_transfer_from_old",
    "is_acc_transfer_to_new",
    "is_role_transfer_from_old",
    "is_role_transfer_to_new",
    "is_valid",

    "role_type_fraud",
    "role_type_no_install_info",
]

common_feature_numeric = [
    "acc_first_purchase_price_usd",
    "acc_role_cnt",
    "acc_total_charge_cnt",
    "acc_total_charge_sum_usd",
    "acc_total_refund_cnt",
    "acc_total_refund_sum_usd",
    "bundle_pop_buy_count_last_day",
    "bundle_pop_buy_count_last_day_campaign_stage_trigger",
    "bundle_pop_buy_count_last_day_tech_unlock_trigger",
    "bundle_pop_buy_count_last_week",
    "bundle_pop_buy_count_last_week_adjust",
    "bundle_pop_buy_max_last_day",
    "bundle_pop_buy_max_last_week",
    "bundle_pop_buy_max_last_week_adjust",
    "bundle_pop_buy_max_price_campaign_stage_trigger",
    "bundle_pop_buy_max_price_campaign_stage_trigger_commax",
    "bundle_pop_buy_max_price_tech_unlock_trigger",
    "bundle_pop_buy_max_price_tech_unlock_trigger_commax",
    "bundle_pop_buy_sum_last_day",
    "bundle_pop_buy_sum_last_week",
    "bundle_pop_buy_sum_last_week_adjust",
    "bundle_pop_count_campaign_stage_trigger_9999",
    "bundle_pop_count_last_day",
    "bundle_pop_count_last_day_campaign_stage_trigger",
    "bundle_pop_count_last_day_tech_unlock_trigger",
    "bundle_pop_count_last_week",
    "bundle_pop_count_last_week_adjust",
    "bundle_pop_count_tech_unlock_trigger_9999",

    "level",
    "max_role_level",
    "money_flow_in_cout_last_day",
    "money_flow_in_cout_last_week",
    "money_flow_in_cout_last_week_adjust",
    "money_flow_out_cout_last_day",
    "money_flow_out_cout_last_week",
    "money_flow_out_cout_last_week_adjust",
    "purchase_price_1499_last_week",
    "purchase_price_1499_last_week_adjust",
    "purchase_price_1999_last_week",
    "purchase_price_1999_last_week_adjust",
    "purchase_price_2999_last_week",
    "purchase_price_2999_last_week_adjust",
    "purchase_price_4999_last_week",
    "purchase_price_4999_last_week_adjust",
    "purchase_price_499_last_week",
    "purchase_price_499_last_week_adjust",
    "purchase_price_9999_last_week",
    "purchase_price_9999_last_week_adjust",
    "purchase_price_999_last_week",
    "purchase_price_999_last_week_adjust",
    "purchase_price_max",
    "purchase_price_max_last_3days",
    "purchase_price_max_last_day",
    "purchase_price_max_last_month",
    "purchase_price_max_last_week",
    "purchase_price_max_last_week_adjust",
    "purchase_price_sum_last_3days",
    "purchase_price_sum_last_day",
    "purchase_price_sum_last_month",
    "purchase_price_sum_last_week",
    "purchase_price_sum_last_week_adjust",

    
    "role_first_purchase_price_usd",
    "role_last_login_duration",
    "role_last_purchase_price_usd",
    "role_lifespan",
    "role_login_last_now",
    "role_login_secondlast_last",
    "role_login_secondlast_now",
    "role_logout_last_now",
    "role_no",
    "role_purchase_first_last",
    "role_purchase_first_now",
    "role_purchase_last_now",
    "role_total_charge_cnt",
    "role_total_charge_sum_usd",
    "role_total_refund_cnt",
    "role_total_refund_sum_usd",
    
    "vip_level",
    "purchase_price_sum_max_ratio",
]

common_feature_ratio = [
    
    "purchase_price_sum_max_ratio_com2",


    'resource_rarity',
    'speedup_rarity',
    'top_key_rarity',
    'officer_bar_ticket_rarity',
    'common_equipment_exp_consume_levelup_1_diff_num_relatively',
    'common_equipment_exp_consume_blueprint_1_diff_num_relatively',
    'common_equipment_exp_rarity',
    'airforce_equipment_exp_consume_levelup_1_diff_num_relatively',
    'airforce_equipment_exp_consume_blueprint_1_diff_num_relatively',
    'airforce_equipment_exp_rarity',
    'ammunition_consume_levelup_1_diff_num_relatively',
    'ammunition_consume_levelup_10_diff_num_relatively',
    'ammunition_consume_blueprint_1_diff_num_relatively',
    'ammunition_consume_blueprint_10_diff_num_relatively',
    'ammunition_rarity',
    'airforce_ammunition_consume_levelup_1_diff_num_relatively',
    'airforce_ammunition_consume_levelup_10_diff_num_relatively',
    'airforce_ammunition_consume_blueprint_1_diff_num_relatively',
    'airforce_ammunition_consume_blueprint_10_diff_num_relatively',
    'airforce_ammunition_rarity',
    'elements_consume_blueprint_1_diff_num_relatively',
    'elements_consume_blueprint_10_diff_num_relatively',
    'elements_consume_reform_diff_num_relatively',
    'elements_rarity',
    'airforce_elements_consume_blueprint_1_diff_num_relatively',
    'airforce_elements_consume_blueprint_10_diff_num_relatively',
    'airforce_elements_consume_reform_diff_num_relatively',
    'airforce_elements_rarity',
    'replacement_wrench_consume_mechanic_1_diff_num_relatively',
    'replacement_wrench_rarity',
    'officer_exp_consume_lvlup_1_diff_num_relatively',
    'officer_exp_rarity',
    'component_consume_diff_num_relatively',
    'chest_choice_replacement_rarity',

    "buy_rate_last_day",
    "buy_rate_last_day_campaign_stage_trigger",
    "buy_rate_last_day_tech_unlock_trigger",
    "buy_rate_last_week",
]

In [3]:
dataset = pd.read_parquet('../../data/raw/0.0.3.v2.pqt')

seqvocab = dataset.bundle_feature.apply(lambda x: [v for _ in filter(lambda y : y is not None, x.values()) for v in _]) 

vocab = {
    'bundle_feature' : list(set([v for _ in seqvocab for v in _])), 
    'language' : dataset.language.unique().tolist(),
    'region' : dataset.region.unique().tolist(),
    'trigger_type' : dataset.trigger_type.unique().tolist(),
    'svr_region' : ['official','global','ios-ptr']
}   

boxcox_vocab = dict()
logbucket_vocab = dict()


def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1e3
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1e6
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1e9
    else:
        return float(x)

for x in common_feature_numeric:
    array = dataset.role_feature.apply(lambda v : value_to_float(v.get(x)))
    array = array.to_numpy()
    c = array.min()
    c = abs(c) + 1
    _, l = boxcox(array + c, None, None) 
    boxcox_vocab[x] = (l,c)

    minv, maxv = array.min() + c, array.max() + c

    logbucket_vocab[x] = np.exp(np.log(maxv)/40)

d:\Anaconda\envs\tf210\lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
d:\Anaconda\envs\tf210\lib\site-packages\numpy\core\_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [4]:
from lmtk.feature_extraction.constants import FEATURE_MASK, MAX_HASH_SIZE
from cityhash import CityHash64
import numpy as np

price_chain = ['499','999','1499', '1999', '2999','4999', '9999']
p2idx = {p:i for i,p in enumerate(price_chain)}
idx2p = {i:p for i,p in enumerate(price_chain)}

def getdataset(query = 'sid % 10<=7'):
    def get_label_func(x):
        max7pay = x.pop('max7pay')
        max7pay = tf.one_hot(max7pay, len(price_chain))
        return x, max7pay
    
    def padding(x, length = 12):
        x = list(x)
        if x is None:
            return [0] * length
        if len(x) <= length:
            return list(x) + [0] * (length - len(x))
        else:
            return list(x)[:length]

    data1 = dataset.assign(
        sid = lambda x: (x.open_id + x.user_id + x.dt + x.trigger_type).apply(CityHash32),
        max7pay = lambda x: x.max7pay.apply(lambda x : p2idx.get(x))
    ).query(query)

    tensors = ()
    names = []

    for f in ['last1bundle', 'last2bundle', 'last3bundle', 'last4bundle', 'last5bundle']:
        x = data1.bundle_feature\
            .apply(lambda x: x.get(f) if x.get(f) is not None else [])\
            .apply(lambda x: map(lambda y: (CityHash64(y) & FEATURE_MASK) % MAX_HASH_SIZE, x))\
            .apply(lambda x: padding(x, 12))\
            .to_list()
        x = tf.convert_to_tensor(x, dtype = tf.uint64)
        t = tf.data.Dataset.from_tensor_slices(x, name = f)
        tensors += (t,)
        names.append(f)

    for f in common_feature_boolean + common_feature_numeric + common_feature_ratio:
        d = data1.role_feature.apply(lambda x: float(x.get(f,'0') or '0'))
        x = d.fillna(0.0).values
        x = tf.convert_to_tensor(x, dtype = tf.float32)
        t = tf.data.Dataset.from_tensor_slices(x, name = f)
        tensors += (t,)
        names.append(f)

    for f in ['language', 'svr_region']:
        x = data1[f].apply(lambda y: (CityHash64(y) & FEATURE_MASK) % MAX_HASH_SIZE).values
        x = tf.convert_to_tensor(x, dtype = tf.uint64)
        t = tf.data.Dataset.from_tensor_slices(x, name = f)
        tensors += (t,)
        names.append(f)

    for f in ['trigger_type', 'max7pay']:
        x = data1[f].values
        t = tf.data.Dataset.from_tensor_slices(x, name = f)
        tensors += (t,)
        names.append(f)

    return tf.data.Dataset.zip(tensors).map(lambda *x: dict(zip(names, x))).map(get_label_func)

trainset = getdataset('sid % 10<=7')
valset = getdataset('sid % 10>7')

In [5]:
inputs = {
    'cat_feature' : [
        tf.keras.layers.Input(shape = (1), dtype = tf.uint64, name = u)
        if u != 'trigger_type'
        else tf.keras.layers.Input(shape = (1), dtype = tf.string, name = u)
        for u in cat_feature
    ],
    'common_feature_boolean' : [
        tf.keras.layers.Input(shape = (1), dtype = tf.float32, name = u)
        for u in common_feature_boolean
    ],
    'common_feature_numeric' : [
        tf.keras.layers.Input(shape = (1), dtype = tf.float32, name = u)
        for u in common_feature_numeric
    ],
    'common_feature_ratio' : [
        tf.keras.layers.Input(shape = (1), dtype = tf.float32, name = u)
        for u in common_feature_ratio
    ],
    'sequence' : [
        tf.keras.layers.Input(shape = (12, ), dtype = tf.uint64, name = u)
        for u in ['last1bundle', 'last2bundle', 'last3bundle', 'last4bundle', 'last5bundle']
    ],
}

embeddings = {
    'user_profile' : {},
    # 'context' : {},
    'sequence' : {},
}

embedding_size = 8

for x in inputs.get('cat_feature'):
    name = x.name
    if name == 'trigger_type':
        x = tf.keras.layers.StringLookup(vocabulary = vocab[name], oov_token=0)(x)
    else:
        v = map(lambda y: (CityHash64(y) & FEATURE_MASK) % MAX_HASH_SIZE, vocab[name])
        x = tf.keras.layers.IntegerLookup(vocabulary=list(v), mask_value=0)(x)
    x = tf.keras.layers.Embedding(len(vocab[name]) + 1, embedding_size, activity_regularizer= tf.keras.regularizers.L2(1e-4))(x)

    embeddings['user_profile'][name] = x

for x in inputs.get('common_feature_boolean'):
    name = x.name
    x = tf.cast(x, tf.int32, name = f'cast_{name}')
    x = tf.keras.layers.Embedding(2,embedding_size, activity_regularizer= tf.keras.regularizers.L2(1e-4))(x)
    embeddings['user_profile'][name] = x

for x in inputs.get('common_feature_ratio'):
    name = x.name
    
    x1 = tf.keras.layers.Dense(embedding_size)(x)
    x1 = tf.expand_dims(x1, axis = 1)
    
    x2 = tf.clip_by_value(x * 20, 0, 19)
    x2 = tf.keras.layers.Embedding(20, embedding_size, activity_regularizer= tf.keras.regularizers.L2(1e-4))(x2)
    x = x1 + x2
    embeddings['user_profile'][name] = x

for x in inputs.get('common_feature_numeric'):
    name = x.name

    # l = tf.constant(boxcox_vocab[name][0], dtype=tf.float32)
    c = tf.constant(boxcox_vocab[name][1], dtype=tf.float32)
    b = tf.constant(logbucket_vocab[name], dtype=tf.float32)

    # i1 = (tf.math.pow(x+c, l) - 1)/l
    i1 = x
    e1 = tf.keras.layers.Dense(embedding_size, activation='leaky_relu')(i1)
    e1 = tf.expand_dims(e1, axis = 1)
    e1 = tf.keras.layers.BatchNormalization()(e1)

    i2 = tf.math.log(x+c) / tf.math.log(b)
    i2 = tf.clip_by_value(i2, 0, 40 - 1)
    e2 = tf.keras.layers.Embedding(40, embedding_size, activity_regularizer= tf.keras.regularizers.L2(1e-4))(i2)

    x = e1 + e2
    embeddings['user_profile'][name] = x

v = map(lambda y: (CityHash64(y) & FEATURE_MASK) % MAX_HASH_SIZE, vocab['bundle_feature'])
sequence_lookup_layer = tf.keras.layers.IntegerLookup(vocabulary=list(v), oov_token= 0)
sequence_embedding_layer = tf.keras.layers.Embedding(len(vocab['bundle_feature']) + 1, embedding_size, activity_regularizer= tf.keras.regularizers.L2(1e-4))

for x in inputs.get('sequence'):
    name = x.name
    x = sequence_lookup_layer(x)
    x = sequence_embedding_layer(x)
    embeddings['sequence'][name] = x

d:\Anaconda\envs\tf210\lib\site-packages\numpy\core\numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [6]:

class GatedLinearUnit(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.linear = tf.keras.layers.Dense(units)
        self.sigmoid = tf.keras.layers.Dense(units, activation="sigmoid")

    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)

class GatedResidualNetwork(tf.keras.layers.Layer):
    def __init__(self, units, dropout_rate):
        super().__init__()
        self.units = units
        self.elu_dense = tf.keras.layers.Dense(units, activation="elu")
        self.linear_dense = tf.keras.layers.Dense(units)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = tf.keras.layers.BatchNormalization()
        self.project = tf.keras.layers.Dense(units)

    def call(self, inputs):
        x = self.elu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x

class VariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs

class FieldVariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grn = GatedResidualNetwork(units, dropout_rate)
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.Flatten()(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)
        x = self.grn(inputs)
        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs
    
class VariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs


In [7]:
embeddings['user_profile'].values()

dict_values([<KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_1')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_2')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_3')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_4')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_5')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_6')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_7')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_8')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_9')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_10')>, <KerasTensor: shape=(None, 1, 8) dtype=float32 (created by layer 'embedding_11')>, <Ke

In [8]:
y = tf.concat(
    list(embeddings['user_profile'].values()) + list(embeddings['sequence'].values()),
    axis = 1
)

y= tf.keras.layers.BatchNormalization()(y)  
res = FieldVariableSelection(y.shape[1], 32, 0.1)(y)

y = tfutils.layers.attn.BatchNormTransformerEncoder(embedding_size, 8, embedding_size)(y)
y = tfutils.layers.attn.BatchNormTransformerEncoder(embedding_size, 4, embedding_size)(y)
y = tfutils.layers.attn.BatchNormTransformerEncoder(embedding_size, 4, embedding_size)(y)

y = FieldVariableSelection(y.shape[1], 32, 0.1)(y)
y = y + res
y = tf.keras.layers.Flatten()(y)
y = tf.keras.layers.Dense(len(p2idx),'softmax')(y)
model = tf.keras.Model(inputs = [i for field in inputs.values() for i in field], outputs = y)

# x,y = next(iter(trainset.batch(2)))
# pred = model(x)
# tf.math.reduce_any(tf.math.is_inf(pred))


In [9]:
x,y = next(iter(trainset.batch(2)))
pred = model(x)

In [14]:
lr = tf.keras.optimizers.schedules.CosineDecay(1e-3,decay_steps = 500,alpha = 0.1)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr,clipnorm=1.),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.AUC(name = 'auc')]
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_auc',min_delta=1e-3,patience=5,mode='max'),
]

ts = trainset.batch(128).prefetch(tf.data.experimental.AUTOTUNE)
vs = valset.batch(128)
model.fit(ts, validation_data=vs, epochs= 1000, callbacks=callbacks)

Epoch 1/1000


In [ ]:
model.save('../../savedmodel/dynamic_pricing/0.0.3/1/')

INFO:tensorflow:Assets written to: ../../savedmodel/dynamic_pricing/0.0.3/1/assets


INFO:tensorflow:Assets written to: ../../savedmodel/dynamic_pricing/0.0.3/1/assets


In [ ]:

# optimizer=tf.keras.optimizers.Adam(learning_rate=lr,clipnorm=1.)
# loss = tf.keras.losses.BinaryCrossentropy()
# with tf.GradientTape() as tape:
#     loss_value = loss(y, model(x))
# grads = tape.gradient(loss_value, model.trainable_weights)
# optimizer.apply_gradients(zip(grads, model.trainable_weights))


# model(x),y

In [13]:
signature_dict = {
    inp.name : inp.type_spec
    for _ in inputs.values()
    for inp in _
}

pchain = ['4.99','9.99','14.99', '19.99', '29.99','49.99', '99.99']
price_chain_tensor = tf.constant(pchain, dtype=tf.string)

@tf.function(input_signature=[signature_dict])
def serve(x):
    pred = model(x)
    pred = tf.math.argmax(pred, axis = 1)
    pred = tf.gather(price_chain_tensor, pred)
    return pred

import pendulum
ts = pendulum.now().int_timestamp
model.save(f'../../savedmodel/dynamic_pricing/0.0.3/{ts}/', signatures=serve)

INFO:tensorflow:Assets written to: ../../savedmodel/dynamic_pricing/0.0.3/1688726922/assets


INFO:tensorflow:Assets written to: ../../savedmodel/dynamic_pricing/0.0.3/1688726922/assets


In [ ]:
x,y = next(iter(valset.batch(32)))

xx = {}
for f, t in signature_dict.items():
    xx[f] = tf.reshape(tf.cast(x[f], t.dtype), (-1, t.shape[1]))

serve(xx)

<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'4.99', b'19.99', b'4.99', b'99.99', b'9.99', b'19.99', b'9.99',
       b'19.99', b'9.99', b'9.99', b'9.99', b'4.99', b'9.99', b'99.99',
       b'49.99', b'19.99', b'14.99', b'9.99', b'9.99', b'19.99', b'19.99',
       b'19.99', b'99.99', b'99.99', b'9.99', b'4.99', b'9.99', b'99.99',
       b'99.99', b'49.99', b'19.99', b'4.99'], dtype=object)>

In [ ]:

for f, t in signature_dict.items():
    print(f, t.dtype, x[f].dtype)


svr_region <dtype: 'uint64'> <dtype: 'uint64'>
trigger_type <dtype: 'string'> <dtype: 'string'>
language <dtype: 'uint64'> <dtype: 'uint64'>
role_create_os_android <dtype: 'float32'> <dtype: 'float32'>
role_create_os_ios <dtype: 'float32'> <dtype: 'float32'>
if_accounts_first_role <dtype: 'float32'> <dtype: 'float32'>
if_pruchase_active_in_1day <dtype: 'float32'> <dtype: 'float32'>
if_pruchase_active_in_30day <dtype: 'float32'> <dtype: 'float32'>
if_pruchase_active_in_3day <dtype: 'float32'> <dtype: 'float32'>
if_pruchase_active_in_7day <dtype: 'float32'> <dtype: 'float32'>
is_acc_transfer_from_old <dtype: 'float32'> <dtype: 'float32'>
is_acc_transfer_to_new <dtype: 'float32'> <dtype: 'float32'>
is_role_transfer_from_old <dtype: 'float32'> <dtype: 'float32'>
is_role_transfer_to_new <dtype: 'float32'> <dtype: 'float32'>
is_valid <dtype: 'float32'> <dtype: 'float32'>
role_type_fraud <dtype: 'float32'> <dtype: 'float32'>
role_type_no_install_info <dtype: 'float32'> <dtype: 'float32'>
acc_

In [ ]:
# pred = []
# true = []
# for x,y in valset.batch(128):
#     pred.append(serve(x))
#     true.append(y)

# p = tf.concat(pred, -1)
# t = tf.math.argmax(tf.concat(true, 0), 1)
# t = tf.gather(price_chain_tensor, t)

# p = tf.strings.to_number(p)
# t = tf.strings.to_number(t)

# tf.reduce_sum(tf.cast(p < t, tf.float32)) / tf.cast(tf.shape(p)[0], tf.float32)

# y,idx,cnt = tf.unique_with_counts(p)
# dist_normal = dict(zip(y.numpy(),cnt.numpy()/cnt.numpy().sum()))
# dist_normal